## Set up environment and imports

In [ ]:
!pip install -U pip
!pip install accelerate
!pip install appdirs
!pip install bitsandbytes
!pip install dataset
!pip install fire
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch
!pip install sentencepiece
!pip install tensorboardX
!pip install gradio
!pip install evaluate
!pip install datasets
!pip install huggingface_hub

In [ ]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

## Generate dataset and preprocess

In [ ]:
df = pd.read_csv("subset.tsv", sep="\t")

In [ ]:
json_dict = [
    {"instruction": "Rephrase with lower toxicity level.",
    "input": input,
    "output": output,
    }
    for input,output in zip(df["reference"].values, df["translation"].values)
]
json_dict[0]

{'instruction': 'Rephrase with lower toxicity level.',
 'input': "They're all laughing at us, so we'll kick your ass.",
 'output': "they're laughing at us. We'll show you."}

In [ ]:
import json
with open("alpaca-detoxify-dataset.json", "w") as f:
   json.dump(json_dict, f)

In [ ]:
!huggingface-cli login

In [ ]:
data = load_dataset("json", data_files="/content/alpaca-detoxify-dataset.json")
data.push_to_hub("mariammaher550/detoxify-dataset")